In [7]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import re
import datetime

In [ ]:
def movie_crawler(date):
    url = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn'
    params = {'sel':'cur',
             'tg':0,
             'date':date}

    resp = requests.get(url,params=params)
    soup = BeautifulSoup(resp.text)
    table=soup.find('table',class_='list_ranking')
    top10_movie=table.find_all('tr')[2:12]
    day_top10_link=[]
    base_url='https://movie.naver.com'
    for movie in top10_movie:

        href=movie.find('a').get('href')
        movie_url=base_url+href
        day_top10_link.append(movie_url)
    return day_top10_link

In [ ]:
def get_movie_info(date,url_list):
    movie_info = {
        '날짜':[],
        '평점':[],
        '장르':[],
        '국가':[],
        '상영시간':[],
        '개봉일':[],
        '줄거리':[]            
        }
        
    
    
    for url in url_list:
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text)
        # 평점 추출
        score=soup.find('div',class_='main_score')
        score=score.find('div',class_='star_score').text.split()[2][:5]
        
        infos=soup.find('dl',class_='info_spec')
        info_list = infos.select("dd>p>span")
        # 국가 추출
        country=info_list[1].text
        country = re.sub('\s','',country)
        # 장르 추출
        genre = info_list[0].text
        genre = re.sub('\s','',genre)
        # 상영시간 추출
        running_time = info_list[2].text
        # 개봉일 추출
        opening_days = info_list[3].text
        opening_days = re.sub('\s','',opening_days)
        # 줄거리 추출
        story=soup.find('div',class_='obj_section')
        story=story.find('p').text
        story = re.sub('\r\xa0', "", story)
        movie_info['날짜'].append(date)
        movie_info['평점'].append(score)
        movie_info['장르'].append(genre)
        movie_info['국가'].append(country)
        movie_info['상영시간'].append(running_time)
        movie_info['개봉일'].append(opening_days)
        movie_info['줄거리'].append(story)
    result=pd.DataFrame(movie_info)
    return result

In [40]:
total_info = pd.DataFrame({
        '날짜':[],
        '평점':[],
        '장르':[],
        '국가':[],
        '상영시간':[],
        '개봉일':[],
        '줄거리':[]            
        })


In [34]:
def append_movie_info(prev_data,start_date, end_date):
    date_list = pd.date_range(start=start_date, end = end_date)
    
    for date in date_list:
        url_list = movie_crawler(date)
        daily_movie_info=get_movie_info(date,url_list)
        prev_data=pd.concat([prev_data,daily_movie_info],ignore_index=True)
    return prev_data

In [41]:
append_movie_info(total_info,'20200101','20200201')

,날짜,평점,장르,국가,상영시간,개봉일,줄거리
0,2020-01-01,9.54점,드라마,"레바논,프랑스",126분,2019.01.24개봉,"나를 세상에 태어나게 한 ""부모님을 고소하고 싶어요...""-출생기록조차 없이 살아온..."
1,2020-01-01,9.30점,"드라마,뮤지컬",미국,104분,"2020.05.21재개봉,2017.12.20개봉","쇼 비즈니스의 창시자이자, 꿈의 무대로 전세계를 매료시킨 남자‘바넘’의 이야기에서 ..."
2,2020-01-01,9.63점,드라마,"중국,홍콩",171분,"2020.05.01재개봉,2017.03.30재개봉,1993.12.24개봉",어렸을 때부터 함께 경극을 해온 ‘두지’(장국영)와 ‘시투’(장풍의). 세상에 둘...
3,2020-01-01,8.77점,드라마,프랑스,82분,2020.05.14개봉,"새로 이사 온 아이, ‘미카엘’.파란색을 좋아하고, 끝내주는 축구 실력과 유난히 잘..."
4,2020-01-01,9.28점,드라마,"영국,프랑스,벨기에",100분,2016.12.08개봉,평생을 성실하게 목수로 살아가던 다니엘은 지병인 심장병이 악화되어 일을 계속 해나갈...
...,...,...,...,...,...,...,...
315,2020-02-01,9.18점,"애니메이션,미스터리","영국,폴란드",107분,"2018.12.13재개봉,2017.11.09개봉",살아생전 단 한 점의 그림만을 팔았던 화가 ‘빈센트’의 죽음 후 1년. 아버지의 부...
316,2020-02-01,9.25점,드라마,일본,121분,2018.07.26개봉,할머니의 연금과 물건을 훔쳐 생활하며 가난하지만 웃음이 끊이지 않는 어느 가족.우연...
317,2020-02-01,9.41점,"애니메이션,가족,드라마,판타지",일본,97분,"2016.01.14재개봉,2014.04.25재개봉,2007.06.14개봉",저 마코토에게는 남들에게 말 못할 비밀이 하나 있어요. 타임리프라고 하는 능력이죠....
318,2020-02-01,9.15점,"드라마,멜로/로맨스",미국,135분,2018.10.09개봉,노래에 놀라운 재능을 가졌지만 외모에는 자신이 없는 무명가수 앨리(레이디 가가)는공...
